## Import Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data_small.csv')
print(df.shape)
df.head()

(289551, 17)


,id,campaignName,description,categories,duration,monetaryGoal,country,city,state,date_created,deadline_date,launched_date,state_changed_at,backers_count,usd_pledged,url,target
0,1123889576,mars-on-earth-an-art-residency,Help a fine art photographer continue her proj...,Space Exploration,27,1000,US,Boston,MA,2015-06-24,2015-10-23,2015-09-26,2015-10-23,53,1884,https://www.kickstarter.com/projects/cassandra...,successful
1,1724173143,vulcan-i-rocket-powered-by-3d-printed-engine,Team of undergraduates racing to be the first ...,Space Exploration,30,15000,US,San Diego,CA,2014-05-06,2015-05-21,2015-04-21,2015-05-21,465,21882,https://www.kickstarter.com/projects/105499101...,successful
2,707260502,starscraper-the-next-generation-of-suborbital-...,What if we built a rocket that is better than ...,Space Exploration,31,10000,US,Boston,MA,2014-11-29,2015-01-09,2014-12-09,2015-01-09,294,17176,https://www.kickstarter.com/projects/burpg/sta...,successful
3,497637964,students-building-a-near-space-balloon-with-li...,A group of high school students are building a...,Space Exploration,30,150,US,Mountain View,CA,2014-11-19,2015-11-26,2015-10-27,2015-11-26,45,970,https://www.kickstarter.com/projects/136362214...,successful
4,1546008758,earth-360,Re-inventing the way we look at our planet by ...,Space Exploration,30,7500,US,Fairfield,CT,2012-04-11,2012-09-21,2012-08-22,2012-09-21,28,7576,https://www.kickstarter.com/projects/211370922...,successful


## Encode strings

In [3]:
def create_dictionary(data):
    temp_dict = {}
    for key in data.iteritems():
        if key[1] not in temp_dict.keys():
            temp_dict[key[1]] = len(temp_dict)
    return temp_dict

In [4]:
# use clean df with no duplicates
df = df.copy()

# create dictionaries for string variables
city_dict = create_dictionary(df['city'])
state_dict = create_dictionary(df['state'])
country_dict = create_dictionary(df['country'])
cat_dict = create_dictionary(df['categories'])

# map dictionaries to dataframe
df['categories'] = df['categories'].map(cat_dict)
df['country'] = df['country'].map(country_dict)
df['city'] = df['city'].map(city_dict)
df['state'] = df['state'].map(state_dict)
df['target'] = df['target'].map({'failed':0, 'successful':1})

In [5]:
# join name and description

df['name_description'] = df['campaignName'].apply(lambda x: ' '.join(x.split('-'))) + ' ' + df.description

In [6]:
df.head()

,id,campaignName,description,categories,duration,monetaryGoal,country,city,state,date_created,deadline_date,launched_date,state_changed_at,backers_count,usd_pledged,url,target,name_description
0,1123889576,mars-on-earth-an-art-residency,Help a fine art photographer continue her proj...,0,27,1000,0,0,0,2015-06-24,2015-10-23,2015-09-26,2015-10-23,53,1884,https://www.kickstarter.com/projects/cassandra...,1,mars on earth an art residency Help a fine art...
1,1724173143,vulcan-i-rocket-powered-by-3d-printed-engine,Team of undergraduates racing to be the first ...,0,30,15000,0,1,1,2014-05-06,2015-05-21,2015-04-21,2015-05-21,465,21882,https://www.kickstarter.com/projects/105499101...,1,vulcan i rocket powered by 3d printed engine T...
2,707260502,starscraper-the-next-generation-of-suborbital-...,What if we built a rocket that is better than ...,0,31,10000,0,0,0,2014-11-29,2015-01-09,2014-12-09,2015-01-09,294,17176,https://www.kickstarter.com/projects/burpg/sta...,1,starscraper the next generation of suborbital ...
3,497637964,students-building-a-near-space-balloon-with-li...,A group of high school students are building a...,0,30,150,0,2,1,2014-11-19,2015-11-26,2015-10-27,2015-11-26,45,970,https://www.kickstarter.com/projects/136362214...,1,students building a near space balloon with li...
4,1546008758,earth-360,Re-inventing the way we look at our planet by ...,0,30,7500,0,3,2,2012-04-11,2012-09-21,2012-08-22,2012-09-21,28,7576,https://www.kickstarter.com/projects/211370922...,1,earth 360 Re-inventing the way we look at our ...


In [8]:
df.name_description[0]

'mars on earth an art residency Help a fine art photographer continue her project about space exploration, Mars, and the scientists who are going to make it possible!'

## NLP

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.model_selection import cross_val_score, train_test_split

import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# nlp = spacy.load('en_core_web_md')

In [10]:
# pick features
target = 'target'
features = ['name_description', 'categories', 'duration'
            , 'monetaryGoal', 'country']

# train/test split
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=22)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((231640, 5), (57911, 5), (231640,), (57911,))

In [ ]:
tfidf_vector = TfidfVectorizer(stop_words='english', max_features=10000)

tfidf_vector.fit(X_train['name_description'])
#  max_df=0.95, min_df=0.1,

In [ ]:
x = tfidf_vector.transform(X_train['name_description'])

In [ ]:
train_desc_matrix = x.todense()

In [ ]:
train_desc_matrix_df = pd.DataFrame(train_desc_matrix, columns=tfidf_vector.get_feature_names())

In [ ]:
print(train_desc_matrix_df.shape)
train_desc_matrix_df.head()

In [11]:
# combine train features with nlp features

def nlp_test(train,test):
    
    X_train = train.copy()
    X_test = test.copy()
    
    # vectorizer
    tfidf_vector = TfidfVectorizer(stop_words='english', max_features=5000)

    # tokenize name
    train_name = X_train['name_description']
    train_name = tfidf_vector.fit_transform(train_name)
    test_name = X_test['name_description']
    test_name = tfidf_vector.transform(test_name) 
    
    # create name matrix
    train_name_matrix = train_name.todense()
    test_name_matrix = test_name.todense()
    
    # create df from name matrix
    train_name_matrix_df = pd.DataFrame(train_name_matrix, columns=tfidf_vector.get_feature_names())
    test_name_matrix_df = pd.DataFrame(test_name_matrix, columns=tfidf_vector.get_feature_names())
    
    #get lengths of train and test
    train_len = len(X_train)
    test_len = len(X_test)

    # add id for merge
    X_train['id'] = list(range(train_len))
    train_name_matrix_df['id'] = list(range(train_len))

    X_test['id'] = list(range(test_len))
    test_name_matrix_df['id'] = list(range(test_len))


    # merge train with name matrix
    X_train = X_train.merge(train_name_matrix_df, on='id', how='inner')
    X_test = X_test.merge(test_name_matrix_df, on='id', how='inner')
    
    
    # drop unnecessary columns
    X_train = X_train.drop(columns=['name_description', 'id'])
    X_test = X_test.drop(columns=['name_description', 'id'])
    
    assert len(X_train) == len(train)
    assert len(X_test) == len(test)
    
    return X_train, X_test

In [12]:
X_train_nlp, y_train_nlp = nlp_test(X_train, X_test)

In [13]:
X_train_nlp.head()

,categories,duration,monetaryGoal,country_x,00,000,01,10,100,1000,...,zelda,zen,zero,zine,zodiac,zombie,zombies,zone,zoo,zu
0,3,20,3000,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,61,30,2500,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,151,32,3500,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,110,30,1000,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,124,30,2000,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [20]:
model = RandomForestClassifier(max_depth=)

model.fit(X_train_nlp, y_train)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
model.score(X_train_nlp, y_train)

0.9880763253324124

In [23]:
y_preds = model.predict(y_train_nlp)

In [29]:
score = accuracy_score(y_preds, y_test)

In [31]:
score

0.7201049886895409

In [32]:
import pickle
pickle.dump(model, open('model_rf_tues.pkl', 'wb'))

## Functions for flask app

In [ ]:
def nlp_john(data_df):
    
    df = data_df.copy()
    
    df['name_description'] = df['campaignName'].apply(lambda x: ' '.join(x.split('-'))) + ' ' + df.description
    
    features = ['name_description', 'categories', 'duration'
            , 'monetaryGoal', 'country']
    
    df = df[features]
    
    # vectorizer
    tfidf_vector = TfidfVectorizer(stop_words='english', max_features=5000)

    # tokenize name
    train_text = df['name_description']
    train_text = tfidf_vector.fit_transform(train_text)
    
    # create name matrix
    train_text_matrix = train_text.todense()
    
    # create df from name matrix
    train_text_matrix_df = pd.DataFrame(train_text_matrix, columns=tfidf_vector.get_feature_names())

    # add id for merge
    df['id'] = list(range(len(df)))
    train_text_matrix_df['id'] = list(range(len(df)))

    # merge train with name matrix
    df_merged = df.merge(train_text_matrix_df, on='id', how='inner')
    
    # drop unnecessary columns
    df_merged = df.drop(columns=['name_description', 'id'])
    
    assert len(df_merged) == len(data_df)
    
    return df_merged